# Deterministic Models and Optimization - Assignment 2

## Edit Distance

### The problem
Given strings X and Y, the proposed algorithm calculatres the amount of minimum modifications needed to turn string X into string Y. Furthermore it finds the best option between the possible edit modifications and log which edits were performed.  The amount of necessary modifications is called the 'edit distance' between the two strings. Since the algorithm must provide the optimal solution, the alternative solutions must be compared to choose the minimum.
This algorithm has a wide variety applications, from spellchecking to comparing DNA gene sequences.


This algorithm has two main parts; generating the costs matrix by solving a number of smaller subproblems and tracing back the solution to find the minimum. 
Therefore we start by looking at X[0] and Y[0] and consider the possible routes to take if they do not match:

##### I. Substitution
In this particular case, we substitute the first character of the string Y. After doing this both strings start with the same character, so we will need to compute the edit distance for X[1,…,n] and Y[1,…,m] characters. The final result will therefore be the cost of the substitution of X[0] plus the edit distance of the remaining substrings.

##### II. Insertion
Alternatively, we could also match the first index of both strings by inserting the first character of Y to the beginning of X. The cost for this solution would be the cost of the insertion plus the edit distance of the original string X and Y[1,…,m].

##### III. Deletion
Analogous to the insertion, when deleting, we only get rid of the first character in X (X[0]). In this case after the modification we have to compute the remaining edit distance between X[1,…,m] and the original string Y and add the cost of the deletion for the optimal solution.

These options for the first index are exactly the same problems that we were originally trying to solve; the edit distance between two strings. Therefore we are solving the original problem by solving n+m smaller subproblems. 
Furthermore as seen in the description of each option, after solving one of the subproblems the remaining number of problems will continually decrease until it reaches the end of both strings.

As soon as we reach the base case we can stop. The base case here is reached as soon as both input strings are empty, because then the edit distance between them is 0 and no computation is needed. If one of them is not empty, we will have to solely either remove the number of characters in string X or insert the characters of Y into X.
In any case the edit distance will be equal to the number of characters in the non-empty string.

$$\textrm{if X[1]=Y[1] then D[1,1]=D[0,0]}\\
\textrm{otherwise}\\
D(1,1) = min \begin{cases} D(0,0) + substitutionCost\\
D(1,0) +deletionCost\\
D(0,1) +insertionCost \end{cases}$$

Using dynamic programming we can drastically reduce the time to compute the optimal solution, since by storing the already computed values in a matrix, with index i for X and index j for Y, we will not recurrently solve every subproblem again.
By solving every one of the subproblems and choosing the option with the minimum cost we will eventually choose the minimum solution for every step and reach the optimal solution for the overall problem.

### The matrix
We will compute the costs in a matrix. For the first row of the matrix the costs are the number of deleted characters that would be necessary to convert one string to an empty string. For the first column, this is the number of insertions to convert the string Y to an empty string.
To fill up the matrix we loop through the rows and columns and perform the following computation: If the values of the row and column ARE the same, we select the value in the previous diagonal because since the characters are the same there is nothing to do, which carries no additional cost. 
If the values are NOT the same, then we assign the lowest value from the previous row, column or diagonal, and we add the cost we assign to  deletions, insertions or substitution, in that order. We continue to do this until the matrix is full.

The value in the position [n,m] of the matrix now holds the value for the total (minimum) cost of the transformation of string X into Y. 
In order to register which modifications were done we can now trace back through the matrix by starting at [n,m] and moving to the minimum cost cell of the three neighboring cells and continue to index [0,0].

### Modifed algorithm for weights
To account for the possible differences in the costs of each modification, we can just replace the cost of each edit inside of the algorithm. The real life application for why differing costs for each modification might be needed is eemplified by automatic spelling correction. 
While typing it may seem more likely that a person hits inadvertently a wrong key on the keyboard compared to hitting a key twice instead of once. In that case we could accommodate this by assigning a lower cost to the substitution. 
Likewise by increasing the cost for an unlikely modification we can therefore control for the probabilities of a successful correction.

### Proof of correctness
For proving that this algorithm is correct we check the trivial base cases.
As described before the base case can be observed when at least one string is empty. In this case the solution is trivial;
I) when both strings are empty, the edit distance is 0, since they are the same
ii) when only one string is empty, the edit distance equals the number of characters in the second string (which equals the needed number of insertions or deletions).

By solving the base cases of the problem we fill the first column and first row of our matrix (called D for “Distance”) correctly. The values in the first row will now equal the index of string Y, while the first column in the matrix will match the index of string X. 
Starting now with the value in D(1,1) we will check the three neighboring values (D(0,0), D(1,0), D(0,1)) already filled out in the matrix and compute D(1,1) according to:
$$
\textrm{if X[1]=Y[1] then D[1,1]=D[0,0]}\\
\textrm{otherwise}\\
D(1,1) = min \begin{cases} D(0,0) + substitutionCost\\
D(1,0) +deletionCost\\
D(0,1) +insertionCost \end{cases}$$

Therefore this will give us the minimum possible value for D(1,1). Since this is the recurrence used for filling up all remaining values in the matrix until D(m,n) this algorithm computes the edit distance correctly.

### Time Complexity
The time complexity of this algorithm is defined by the computing cost for each subproblem.
To compute all the values for the matrix we have to solve (m+1)*(n+1) subproblems. Each subproblem can be solved in constant time and therefore the algorithm runs in O(mn) time. 

In [1]:
# Set strings to compare here
DNAX='ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA'
DNAY='TACTAGCTTACTTACCCATCAGGTTTTAGAGATGGCAACCA'

PRX='AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP'
PRY='SGAPGQRGEPGPQGHAGAPGPPGPPGSDG'

import numpy as np
#np.set_printoptions(threshold=np.nan)
# This would allow us to see the full cost matrix instead of just a summary.

def EditDistance(X, Y):
    # Setting up all the working variables. Characters of X and Y in lists and their
    #lenghts. In order to record the necessary modifications to X, we create an empty list.
    X = list(X)
    Y = list(Y)
    lenX = len(X)
    lenY = len(Y)
    mods = []
    # Generate the empty matrix initialized with 0's
    D = np.zeros([lenX+1, lenY+1])
    # costs of deleting all 
    for i in range(lenX):
        D[i+1][0] = D[i][0] + 1
    #costs of inserting all 
    for j in range(lenY):
        D[0][j+1] = D[0][j] + 1
    #costs of each sucessive step
    for i in range(lenX):
        for j in range(lenY):
            # If the characters in the strings match, the cost doesn't increase. No edit.
            if X[i] == Y[j]:
                D[i+1][j+1] = D[i][j]
            # If there is no match, we set the minimum cost for reaching that particular cell.
            else:
                D[i+1][j+1] = min(D[i+1][j] + 1, D[i][j+1] + 1, D[i][j] + 1)
    # print (D,'\n') # to suppress the matrix
    print ('The Edit Distance is', int(D[lenX,lenY]),'.\n')
    c = int(D[lenX,lenY])

    # In order to know which edits are done in each step we need to trace back the cost matrix.
    while lenX > 0 or lenY > 0:
        # We search for the lowest cost in the previous 3 cells.
        cost_S = D[lenX-1][lenY-1]
        cost_D = D[lenX - 1][lenY]
        cost_I = D[lenX][lenY - 1]
        # If the cost of a Substitution is not greater than the rest and it's equal to the current matrix cost...
        if (cost_S == min(cost_S,cost_D,cost_I) and cost_S == D[lenX][lenY]):
            # ... nothing to modifiy. Move to the previous diagonal.
            mods.append('=')
            lenX -= 1
            lenY -= 1
            # If the cost is decreasing, it's a substitution. Move to the previous diagonal.
        elif (cost_S == min(cost_S,cost_D,cost_I) and cost_S != D[lenX][lenY]):
            mods.append('S')
            lenX -= 1
            lenY -= 1
            #If the cost of a Deletion is the smallest, perform a deletion...
        elif (cost_D == min(cost_S,cost_D,cost_I)):
            mods.append('D')
            Y.insert(lenY, '_') 
            # ... and move to the previous row.
            lenX -= 1
            #If the cost of a Deletion is the smallest, perform an insertion...
        elif (cost_I == min(cost_S,cost_D,cost_I)):
            mods.append('I')
            X.insert(lenX, '_') 
            # ... and move to the previous column.
            lenY -= 1
        else:
            print ('error')
    # Print the appended string in inverse (original) order.
    print (' X:[',''.join(X),']\n', 'Y:[',''.join(Y),']\n', 'E:[',''.join(mods[::-1]),']','\n')
    print('The number of operations is', sum(1 for x in mods if (x != '=')),'.\n')
    # return ''
    return c

print ('DNA \n')
print (EditDistance(DNAX,DNAY))
print ('Proteins \n')
print (EditDistance(PRX,PRY))


DNA 

The Edit Distance is 10 .

 X:[ ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA ]
 Y:[ __TACTAGCTTACTTACCCATCAGGT__TTTAGAG_ATGGCAACCA ]
 E:[ DD======S========SS=======DD=======DS=S======= ] 

The number of operations is 10 .

10
Proteins 

The Edit Distance is 37 .

 X:[ AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP ]
 Y:[ __S____GAPGQRGEPGPQGHAGAPGPPGPPGS________D___G____ ]
 E:[ DD=DDDD=S=S=SSSSSS=SS=SS=S=SS==S=DDDDDDDD=DDD=DDDD ] 

The number of operations is 37 .

37


**2**. Modify the previous algorithm with a penalty cost function: operations D and I have unit cost 2, whereas operation S has unit cost 1.

### Levenshtein Distance
The case of Edit Distance having different costs for each modification is called the Levenshtein Distance. In the algorithm, we substitute the constantly increasing cost by a variable which will contain the specific cost for each modification.

In [2]:
# Set strings to compare here
DNAX='ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA'
DNAY='TACTAGCTTACTTACCCATCAGGTTTTAGAGATGGCAACCA'

PRX='AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP'
PRY='SGAPGQRGEPGPQGHAGAPGPPGPPGSDG'

import numpy as np
#np.set_printoptions(threshold=np.nan) #would print the full cost matrix.

def EditDistanceLev(X, Y, DEL=2,INS=2, SBS=1):
    # Setting up all the working variables. Characters of X and Y in lists and their lenghts. 
    # In order to record the necessary modifications to B, we create an empty list.
    X = list(X)
    Y = list(Y)
    lenX = len(X)
    lenY = len(Y)
    mods = []
    # Generate the empty matrix initialized with 0's
    D = np.zeros([lenX+1, lenY+1])
    # costs of deleting all Y
    for i in range(lenX):
        D[i+1][0] = D[i][0] + DEL
    # costs of inserting all X
    for j in range(lenY):
        D[0][j+1] = D[0][j] + INS
    # costs of each sucessive step
    for i in range(lenX):
        for j in range(lenY):
            # If the characters in the strings match, the cost doesn't increase. No edit.
            if X[i] == Y[j]:
                D[i+1][j+1] = D[i][j]
            # If there is no match, we set the minimum cost for reaching that particular cell.
            else:
                D[i+1][j+1] = min(D[i+1][j] + DEL, D[i][j+1] + INS, D[i][j] + SBS)
    # print (D,'\n') # would print part of the matrix
    print ('The Levenshtein Distance is', int(D[lenX,lenY]),'.\n')
    c = int(D[lenX,lenY])

    # In order to know which edits are done in each step we need to trace back the cost matrix.
    while lenX > 0 or lenY > 0:
        # We search for the lowest cost in the previous 3 cells.
        cost_S = D[lenX-1][lenY-1]
        cost_D = D[lenX - 1][lenY]
        cost_I = D[lenX][lenY - 1]
        if (cost_S+SBS == min(cost_S+SBS,cost_D+DEL,cost_I+INS) and cost_S == D[lenX][lenY]):
            mods.append('=')
            lenX -= 1
            lenY -= 1
        elif (cost_S+SBS == min(cost_S+SBS,cost_D+DEL,cost_I+INS) and cost_S != D[lenX][lenY]):
            mods.append('S')
            lenX -= 1
            lenY -= 1
        elif (cost_D+DEL == min(cost_S+SBS,cost_D+DEL,cost_I+INS)):
            mods.append('D')
            Y.insert(lenY, '_') 
            # ... and move to the previous row.
            lenX -= 1
        elif (cost_I+INS == min(cost_S+SBS,cost_D+DEL,cost_I+INS)):
            mods.append('I')
            X.insert(lenX, '_') 
            # ... and move to the previous column.
            lenY -= 1
        else:
            print ('error')

    # Print the appended strings in inverse (original) order.
    print (' X:[',''.join(X),']\n', 'Y:[',''.join(Y),']\n', 'E:[',''.join(mods[::-1]),']','\n')
    print('The number of operations is ', sum(1 for x in mods if (x != '=')),'.\n')
    
    return c

print ('DNA \n')
print (EditDistanceLev(DNAX,DNAY))
print ('Proteins \n')
print (EditDistanceLev(PRX,PRY))

DNA 

The Levenshtein Distance is 15 .

 X:[ ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA ]
 Y:[ __TACTAGCTTACTTACCCATCAGGT__TTTAGAG_ATGGCAACCA ]
 E:[ DD======S========SS=======DD=======DS=S======= ] 

The number of operations is  10 .

15
Proteins 

The Levenshtein Distance is 58 .

 X:[ AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP ]
 Y:[ __S____GAPGQRGEPGPQGHAGAPGPPGPPGS________D___G____ ]
 E:[ DD=DDDD=S=S=SSSSSS=SS=SS=S=SS==S=DDDDDDDD=DDD=DDDD ] 

The number of operations is  37 .

58


### Exercise 4

In [3]:
import random
import string
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

### Function to generate the random words of length 50

We start by defining the function random_input which creates a matrix whose entries are computed by sampling letters from the alphabet {A,B,C,D} with a given probability assigned to each letter (by default all equal to ¼). We use random_input to build two matrices (X and Y) where the number of rows represents the number of words we want to compare (N = 100, 500, 1000) and the number of columns represents the number of letters in each word (L = 50). We create three vectors that collect the edit distance over every row of X and Y for the different numbers of word in the three samples. The histograms show the empirical distributions of the computed edit distances with sample mean and sample variance, together with a normal density of the same parameters.

In [4]:
def random_input(rows,cols,prob=[1/4,1/4,1/4,1/4]):
    letters = string.ascii_uppercase[0:4] # select alphabeth {A,B,C,D} 
                                        #from string.ascii_uppercase
    x = np.empty((rows,cols), dtype='str') # create an empty matrix of type 'string'
    for i in range(rows): # for every row
        for j in range(cols): # for every column
            x[i,j] = np.random.choice(list(letters), p=prob) 
            # assign randomly a letter from the alphabet {A,B,C,D} to x_ij 
            # where the probabilities are specified by prob (default is uniform)
    return x

### Create 3 vectors of *Edit Distances* for 100, 500 and 1000 words of length 50 with uniform distribution

In [5]:
%%capture
# the above magic is used to suppress the output

for N in [100, 500, 1000]: # number of random words
    L = 50 # number of letters in each word
    X = random_input(N,L) # matrix collecting the first N input words
    Y = random_input(N,L) # matrix collecting the second N input words
    if (N==100):
        # vector to store the computed edit distances for N = 100
        store_EditDistances_100 = np.zeros(N)
        for i in range(N): # for each of the N words
            # compute the edit distance between the i-th word in X and Y
            store_EditDistances_100[i] = EditDistance(X[i,:],Y[i,:]) 
    elif (N==500):
        store_EditDistances_500 = np.zeros(N) 
        # vector to store the computed edit distances for N = 500
        for i in range(N): # for each of the N words
            # compute the edit distance between the i-th word in X and Y
            store_EditDistances_500[i] = EditDistance(X[i,:],Y[i,:]) 
    else:
        # vector to store the computed edit distances for N = 1000
        store_EditDistances_1000 = np.zeros(N)
        for i in range(N): # for each of the N words
            # compute the edit distance between the i-th word in X and Y
            store_EditDistances_1000[i] = EditDistance(X[i,:],Y[i,:]) 

### Histograms for N = 100; 500 and 1000

<img src="edit_dist_hist.png">

### Non uniform probabilities
The function random_input accomodates for diffent probabilities as long as they sum up to 1. We can sample $(p_1, p_2, p_3, p_4)$ using the following algorithm and then plug them as input of the created function random_input.

In [7]:
p1 = random.uniform(0,1)
p2 = random.uniform(0,1-p1)
p3 = random.uniform(0,1-p1-p2)
p4 = 1-p1-p2-p3 
prob = [p1,p2,p3,p4] # the input of the function random_input
print(prob)
sum(prob)

[0.9809808076958625, 0.010099936212038045, 0.007590575156229062, 0.0013286809358704034]


1.0

In [8]:
# test:
print(random_input(1,10,[1,0,0,0]))
print(random_input(1,10,[0.5,0,0,0.5]))
print(random_input(1,10,prob))

[['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A']]
[['A' 'A' 'A' 'A' 'A' 'D' 'D' 'D' 'A' 'D']]
[['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A']]


### References
Schulz, Klaus U.; Mihov, Stoyan (2002). "Fast string correction with Levenshtein automata". International Journal of Document Analysis and Recognition. 5 (1): 67–85

Navarro, Gonzalo (1 March 2001). "A guided tour to approximate string matching" (PDF). ACM Computing
Surveys. 33 (1): 31–88. doi:10.1145/375360.375365. Retrieved 19 March2015.

R Wagner; M. Fischer (1974). "The string-to-string correction problem". J. ACM. 21: 168–178

"Algorithms for approximate string matching" (PDF). Information and Control. 64 (1–3): 100–118. 1985